##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompting Engineering

Welcome to the Kaggle 5-day Generative AI course!

This notebook will show you how to get started with the Gemini API and walk you through some of the example prompts and techniques that you can also read about in the Prompting whitepaper. You don't need to read the whitepaper to use this notebook, but the papers will give you some theoretical context and background to complement this interactive notebook.

### Install the SDK

In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

Import the SDK and some helpers for rendering the output.

In [2]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

Set up a retry helper. This allows you to "Run all" without worrying about per-minute quota.

In [3]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

### Run your first prompt

In this step, you will test that your API key is set up correctly by making a request.

The Python SDK uses a [`Client` object](https://googleapis.github.io/python-genai/genai.html#genai.client.Client) to make requests to the API. The client lets you control which back-end to use (between the Gemini API and Vertex AI) and handles authentication (the API key).

The `gemini-2.0-flash` model has been selected here.

**Note**: If you see a `TransportError` on this step, you may need to **🔁 Factory reset** the notebook one time.

In [5]:
client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents="Explain AI to me like I'm a kid.")

print(response.text)

Imagine your brain is like a super-smart toy! This toy can learn and do amazing things, just like you. That's kind of what **Artificial Intelligence (AI)** is all about!

Think about these things:

*   **Learning New Stuff:** Remember when you learned your ABCs or how to ride a bike? You practiced and practiced until you got good at it, right? AI is like a computer that can **learn** new things by looking at lots and lots of examples.

    *   **Example:** If you show a computer millions of pictures of cats, it can learn what a cat looks like. Then, if you show it a new picture, it can tell you, "Hey, that's a cat!"

*   **Solving Problems:** Sometimes you have to figure out puzzles or how to build the tallest tower with blocks. AI can also be programmed to **solve problems**.

    *   **Example:** Your favorite video game might have characters that seem to know what to do. That's AI helping them make decisions! Or, a map app on your parent's phone uses AI to find the quickest way to g

The response often comes back in markdown format, which you can render directly in this notebook.

In [6]:
Markdown(response.text)

Imagine you have a super smart toy robot! This robot isn't like a regular toy that just does one thing. This robot can learn and think, a little bit like you do!

**What is AI?**

AI stands for **Artificial Intelligence**. "Artificial" means something that's made by people, not grown like a flower or born like a baby. "Intelligence" means being able to learn, understand, and solve problems.

So, AI is like making computers and robots **smart** enough to do things that usually only people can do!

**How does it learn?**

Think about how you learn to ride a bike. At first, you might fall a lot, right? But your brain remembers what made you fall and tries to do it differently next time. You learn to balance, steer, and pedal better.

AI learns in a similar way, but instead of falling off a bike, it looks at lots and lots of information.

*   **Looking at Pictures:** Imagine showing your robot thousands of pictures of cats. After seeing so many cats, the robot starts to notice what makes a cat a cat: pointy ears, whiskers, a tail. Then, if you show it a new picture, it can say, "Hey, that looks like a cat!"
*   **Listening to Sounds:** If you play lots of different animal sounds for the robot, it can learn to tell the difference between a dog barking and a cat meowing.
*   **Reading Stories:** If you read the robot many stories, it can start to understand what words mean and how they fit together to tell a story.

**What can AI do?**

AI is already helping us in many cool ways!

*   **Talking to you:** When you talk to a smart speaker like Alexa or Google Assistant, that's AI! It understands your words and tries to answer your questions or play your favorite songs.
*   **Playing Games:** Some video games have characters that are controlled by AI. They can be very smart and challenging to play against!
*   **Helping Doctors:** AI can look at X-rays and help doctors find things that might be hard to see.
*   **Making your phone smarter:** When your phone suggests the next word you might want to type, or when it can recognize faces in your photos, that's AI at work!
*   **Self-driving cars:** Imagine a car that can drive itself! That's a big and exciting use of AI.

**Is AI like a real brain?**

Not exactly. A real brain is amazing and can do so many things we don't even understand yet. AI is good at specific tasks, like recognizing pictures or playing chess. It's like a very, very clever tool that helps us do things faster or in new ways.

**So, AI is like having smart helpers that can learn and do amazing things!** It's like magic, but it's made by people who are very good at building clever computer programs.

### Start a chat

The previous example uses a single-turn, text-in/text-out structure, but you can also set up a multi-turn chat structure too.

In [7]:
chat = client.chats.create(model='gemini-2.5-flash-lite', history=[])
response = chat.send_message('Hello! My name is Zlork.')
print(response.text)

Hello Zlork! It's nice to meet you. How can I help you today?


In [8]:
response = chat.send_message('Can you tell me something interesting about dinosaurs?')
print(response.text)

Absolutely, Zlork! Here's something I find pretty fascinating about dinosaurs:

**Did you know that some dinosaurs had feathers, and were actually closely related to modern birds?**

For a long time, our image of dinosaurs was heavily influenced by fossils that lacked soft tissue. This led to the idea of scaly, lizard-like creatures. However, over the past few decades, a remarkable number of feathered dinosaur fossils have been discovered, particularly in China.

These discoveries have shown us that many theropod dinosaurs, the group that includes iconic species like *Velociraptor* and the mighty *Tyrannosaurus rex*, had feathers. These feathers varied in complexity, from simple, hair-like structures to more elaborate, vaned feathers similar to those on modern birds.

This evidence strongly suggests that birds are direct descendants of feathered dinosaurs. So, in a way, when you see a bird flying around today, you're looking at a modern-day dinosaur! It really bridges the gap between t

While you have the `chat` object alive, the conversation state
persists. Confirm that by asking if it knows the user's name.

In [9]:
response = chat.send_message('Do you remember what my name is?')
print(response.text)

Yes, Zlork! I remember your name is Zlork. It was nice meeting you.


### Choose a model

The Gemini API provides access to a number of models from the Gemini model family. Read about the available models and their capabilities on the [model overview page](https://ai.google.dev/gemini-api/docs/models/gemini).

In this step you'll use the API to list all of the available models.

In [10]:
for model in client.models.list():
  print(model.name)

models/embedding-gecko-001
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/g

The [`models.list`](https://ai.google.dev/api/models#method:-models.list) response also returns additional information about the model's capabilities, like the token limits and supported parameters.

In [11]:
from pprint import pprint

for model in client.models.list():
  if model.name == 'models/gemini-2.0-flash':
    pprint(model.to_json_dict())
    break

{'description': 'Gemini 2.0 Flash',
 'display_name': 'Gemini 2.0 Flash',
 'input_token_limit': 1048576,
 'name': 'models/gemini-2.0-flash',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent',
                       'countTokens',
                       'createCachedContent',
                       'batchGenerateContent'],
 'tuned_model_info': {},
 'version': '2.0'}


## Explore generation parameters



### Output length

When generating text with an LLM, the output length affects cost and performance. Generating more tokens increases computation, leading to higher energy consumption, latency, and cost.

To stop the model from generating tokens past a limit, you can specify the `max_output_tokens` parameter when using the Gemini API. Specifying this parameter does not influence the generation of the output tokens, so the output will not become more stylistically or textually succinct, but it will stop generating tokens once the specified length is reached. Prompt engineering may be required to generate a more complete output for your given limit.

In [12]:
from google.genai import types

short_config = types.GenerateContentConfig(max_output_tokens=200)

response = client.models.generate_content(
    model='gemini-2.5-flash-lite',
    config=short_config,
    contents='Write a 1000 word essay on the importance of olives in modern society.')

print(response.text)

## The Enduring Significance of the Olive in Modern Society

The humble olive, a small fruit from a tree whose gnarled branches have witnessed millennia of human history, occupies a surprisingly significant and multifaceted position in modern society. Beyond its ancient lineage, the olive has transcended its origins to become a cornerstone of global cuisine, a symbol of health and wellness, an economic engine for numerous regions, and a potent cultural icon. Its importance is not merely historical but deeply embedded in the fabric of our contemporary lives, shaping our diets, our economies, and even our understanding of a healthy lifestyle.

At the forefront of the olive's modern relevance lies its indispensable role in global gastronomy. The Mediterranean diet, lauded worldwide for its health benefits, is fundamentally built upon a foundation of olive oil. This liquid gold, extracted from the flesh of the olive, is more than just a cooking medium; it is a flavor enhancer, a textural m

In [13]:
response = client.models.generate_content(
    model='gemini-2.5-flash-lite',
    config=short_config,
    contents='Write a short poem on the importance of olives in modern society.')

print(response.text)

From ancient groves, a verdant prize,
That graces tables, fills our eyes.
A sun-kissed sphere, a briny gem,
More than a snack, a diadem.

In oils that shimmer, rich and fine,
A culinary cornerstone, divine.
A bitter bite, a nuanced grace,
That elevates each dining space.

From health's embrace to flavors bold,
A story in its flesh, untold.
The humble olive, small and round,
In modern life, profoundly found.


Explore with your own prompts. Try a prompt with a restrictive output limit and then adjust the prompt to work within that limit.

### Temperature

Temperature controls the degree of randomness in token selection. Higher temperatures result in a higher number of candidate tokens from which the next output token is selected, and can produce more diverse results, while lower temperatures have the opposite effect, such that a temperature of 0 results in greedy decoding, selecting the most probable token at each step.

Temperature doesn't provide any guarantees of randomness, but it can be used to "nudge" the output somewhat.

In [14]:
high_temp_config = types.GenerateContentConfig(temperature=2.0)

for _ in range(5):
  response = client.models.generate_content(
      model='gemini-2.5-flash-lite',
      config=high_temp_config,
      contents='Pick a random colour... (respond in a single word)')

  if response.text:
    print(response.text, '-' * 25)

Indigo -------------------------
Emerald -------------------------
Teal -------------------------
Azure -------------------------
Turquoise -------------------------


Now try the same prompt with temperature set to zero. Note that the output is not completely deterministic, as other parameters affect token selection, but the results will tend to be more stable.

In [15]:
low_temp_config = types.GenerateContentConfig(temperature=0.0)

for _ in range(5):
  response = client.models.generate_content(
      model='gemini-2.5-flash-lite',
      config=low_temp_config,
      contents='Pick a random colour... (respond in a single word)')

  if response.text:
    print(response.text, '-' * 25)

Azure -------------------------
Azure -------------------------
Azure -------------------------
Azure -------------------------
Azure -------------------------


### Top-P

Like temperature, the top-P parameter is also used to control the diversity of the model's output.

Top-P defines the probability threshold that, once cumulatively exceeded, tokens stop being selected as candidates. A top-P of 0 is typically equivalent to greedy decoding, and a top-P of 1 typically selects every token in the model's vocabulary.

You may also see top-K referenced in LLM literature. Top-K is not configurable in the Gemini 2.0 series of models, but can be changed in older models. Top-K is a positive integer that defines the number of most probable tokens from which to select the output token. A top-K of 1 selects a single token, performing greedy decoding.


Run this example a number of times, change the settings and observe the change in output.

In [16]:
model_config = types.GenerateContentConfig(
    temperature=1.0,
    top_p=0.95,
)

story_prompt = "You are a creative writer. Write a short story about a cat who goes on an adventure."
response = client.models.generate_content(
    model='gemini-2.5-flash-lite',
    config=model_config,
    contents=story_prompt)

print(response.text)

Whiskerton III, or Whisker as he preferred, was a creature of refined habit. His days revolved around the sunbeams that dappled his favorite armchair, the precise cadence of kibble hitting his ceramic bowl, and the strategic napping locations that offered optimal warmth and minimal disturbance. Yet, beneath his placid exterior, a flicker of something wild stirred.

It began with a dream. A dream of emerald forests, of chattering squirrels that weren’t safely behind glass, and of a moon so vast it seemed to whisper secrets only to him. Whisker awoke with a twitch of his tail and a rumble in his chest that was more than just contentment.

The catalyst for his adventure arrived on a Tuesday, disguised as a rogue butterfly. It was a creature of improbable color, with wings like stained glass, that dared to flit past his windowpane, taunting him with its freedom. Whisker’s hunter’s instinct, long dormant, flared. He paced, he yowled, he batted at the air with uncharacteristic ferocity. The 

## Prompting

This section contains some prompts from the chapter for you to try out directly in the API. Try changing the text here to see how each prompt performs with different instructions, more examples, or any other changes you can think of.

### Zero-shot

Zero-shot prompts are prompts that describe the request for the model directly.

In [17]:
model_config = types.GenerateContentConfig(
    temperature=0.1,
    top_p=1,
    max_output_tokens=5,
)

zero_shot_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response = client.models.generate_content(
    model='gemini-2.5-flash-lite',
    config=model_config,
    contents=zero_shot_prompt)

print(response.text)

Sentiment: POSITIVE


#### Enum mode

The models are trained to generate text, and while the Gemini 2.0 models are great at following instructions, other models can sometimes produce more text than you may wish for. In the preceding example, the model will output the label, but sometimes it can include a preceding "Sentiment" label, and without an output token limit, it may also add explanatory text afterwards. See [this prompt in AI Studio](https://aistudio.google.com/prompts/1gzKKgDHwkAvexG5Up0LMtl1-6jKMKe4g) for an example.

The Gemini API has an [Enum mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Enum.ipynb) feature that allows you to constrain the output to a fixed set of values.

In [23]:
import enum

class Sentiment(enum.Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"


response = client.models.generate_content(
    model='gemini-2.5-flash',
    config=types.GenerateContentConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ),
    contents=zero_shot_prompt)

print(response.text)

positive


When using constrained output like an enum, the Python SDK will attempt to convert the model's text response into a Python object automatically. It's stored in the `response.parsed` field.

In [ ]:
enum_response = response.parsed
print(enum_response)
print(type(enum_response))

### One-shot and few-shot

Providing an example of the expected response is known as a "one-shot" prompt. When you provide multiple examples, it is a "few-shot" prompt.

In [24]:
few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "pepperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}
```

ORDER:
"""

customer_order = "Give me a large with cheese & pineapple"

response = client.models.generate_content(
    model='gemini-2.5-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ),
    contents=[few_shot_prompt, customer_order])

print(response.text)

```json
{
"size": "large",
"type": "normal",
"ingredients": ["cheese", "pineapple"]
}
```


#### JSON mode

To provide control over the schema, and to ensure that you only receive JSON (with no other text or markdown), you can use the Gemini API's [JSON mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb). This forces the model to constrain decoding, such that token selection is guided by the supplied schema.

In [25]:
import typing_extensions as typing

class PizzaOrder(typing.TypedDict):
    size: str
    ingredients: list[str]
    type: str


response = client.models.generate_content(
    model='gemini-2.5-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ),
    contents="Can I have a large dessert pizza with apple and chocolate")

print(response.text)

{"size":"large","ingredients":["apple","chocolate"],"type":"dessert pizza"}


### Chain of Thought (CoT)

Direct prompting on LLMs can return answers quickly and (in terms of output token usage) efficiently, but they can be prone to hallucination. The answer may "look" correct (in terms of language and syntax) but is incorrect in terms of factuality and reasoning.

Chain-of-Thought prompting is a technique where you instruct the model to output intermediate reasoning steps, and it typically gets better results, especially when combined with few-shot examples. It is worth noting that this technique doesn't completely eliminate hallucinations, and that it tends to cost more to run, due to the increased token count.

Models like the Gemini family are trained to be "chatty" or "thoughtful" and will provide reasoning steps without prompting, so for this simple example you can ask the model to be more direct in the prompt to force a non-reasoning response. Try re-running this step if the model gets lucky and gets the answer correct on the first try.

In [26]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
am 20 years old. How old is my partner? Return the answer directly."""

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt)

print(response.text)

28


Now try the same approach, but indicate to the model that it should "think step by step".

In [27]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now,
I am 20 years old. How old is my partner? Let's think step by step."""

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt)

Markdown(response.text)

Let's break it down step-by-step:

1.  **When you were 4 years old:**
    *   Your age = 4
    *   Your partner's age = 3 * your age = 3 * 4 = 12 years old.

2.  **Find the age difference:**
    *   The difference in age between you and your partner is constant.
    *   Age difference = Partner's age - Your age = 12 - 4 = 8 years.

3.  **Calculate your partner's current age:**
    *   Now, your age = 20
    *   Since the age difference is 8 years, your partner is still 8 years older than you.
    *   Partner's current age = Your current age + Age difference = 20 + 8 = 28 years old.

Your partner is **28 years old**.

### ReAct: Reason and act

In this example you will run a ReAct prompt directly in the Gemini API and perform the searching steps yourself. As this prompt follows a well-defined structure, there are frameworks available that wrap the prompt into easier-to-use APIs that make tool calls automatically, such as the LangChain example from the "Prompting" whitepaper.

To try this out with the Wikipedia search engine, check out the [Searching Wikipedia with ReAct](https://github.com/google-gemini/cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.ipynb) cookbook example.

In [29]:
model_instructions = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation,
Observation is understanding relevant information from an Action's output and Action can be one of three types:
 (1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it
     will return some similar entities to search and you can try to search the information from those topics.
 (2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches,
     so keep your searches short.
 (3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

example1 = """Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>
"""

example2 = """Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>
"""

# Come up with more examples yourself, or take a look through https://github.com/ysymyth/ReAct/

To capture a single step at a time, while ignoring any hallucinated Observation steps, you will use `stop_sequences` to end the generation process. The steps are `Thought`, `Action`, `Observation`, in that order.

In [30]:
question = """Question
Who was the youngest author listed on the transformers NLP paper?
"""

# You will perform the Action; so generate up to, but not including, the Observation.
react_config = types.GenerateContentConfig(
    stop_sequences=["\nObservation"],
    system_instruction=model_instructions + example1 + example2,
)

# Create a chat that has the model instructions and examples pre-seeded.
react_chat = client.chats.create(
    model='gemini-2.5-flash',
    config=react_config,
)

resp = react_chat.send_message(question)
print(resp.text)

Action 1
<search>Attention Is All You Need authors</search>



Now you can perform this research yourself and supply it back to the model.

In [31]:
observation = """Observation 1
[1706.03762] Attention Is All You Need
Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.
"""
resp = react_chat.send_message(observation)
print(resp.text)

Thought 1
The question asks for the youngest author listed on the Transformers NLP paper. I have the list of authors. I need to find their birth dates or ages to determine the youngest. Since I cannot directly search for birth dates of all authors at once, I will start by searching for each author. I will try to find their birthdates or an indication of their age.

Action 1
<search>Aidan N. Gomez birth date</search>


This process repeats until the `<finish>` action is reached. You can continue running this yourself if you like, or try the [Wikipedia example](https://github.com/google-gemini/cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.ipynb) to see a fully automated ReAct system at work.

## Thinking mode

The experiemental Gemini Flash 2.0 "Thinking" model has been trained to generate the "thinking process" the model goes through as part of its response. As a result, the Flash Thinking model is capable of stronger reasoning capabilities in its responses.

Using a "thinking mode" model can provide you with high-quality responses without needing specialised prompting like the previous approaches. One reason this technique is effective is that you induce the model to generate relevant information ("brainstorming", or "thoughts") that is then used as part of the context in which the final response is generated.

Note that when you use the API, you get the final response from the model, but the thoughts are not captured. To see the intermediate thoughts, try out [the thinking mode model in AI Studio](https://aistudio.google.com/prompts/new_chat?model=gemini-2.0-flash-thinking-exp-01-21).

In [10]:
import io
from IPython.display import Markdown, clear_output


response = client.models.generate_content_stream(
    model='gemini-2.5-flash',
    contents='Who was the youngest author listed on the transformers NLP paper?',
)

buf = io.StringIO()
for chunk in response:
    buf.write(chunk.text)
    # Display the response as it is streamed
    print(chunk.text, end='')

# And then render the finished response as formatted markdown.
clear_output()
Markdown(buf.getvalue())

The youngest author listed on the "Attention Is All You Need" paper (the "Transformers NLP paper") was **Aidan N. Gomez**.

He was an undergraduate student (or just starting his PhD) at the University of Toronto when he co-authored the paper. He was around 20 or 21 years old when the paper was published in 2017, making him considerably younger than the other more senior researchers at Google Brain.

## Code prompting

### Generating code

The Gemini family of models can be used to generate code, configuration and scripts. Generating code can be helpful when learning to code, learning a new language or for rapidly generating a first draft.

It's important to be aware that since LLMs can make mistakes, and can repeat training data, it's essential to read and test your code first, and comply with any relevant licenses.

In [9]:
# The Gemini models love to talk, so it helps to specify they stick to the code if that
# is all that you want.
code_prompt = """
Write a Python function to calculate the factorial of a number. No explanation, provide only the code.
"""

response = client.models.generate_content(
    model='gemini-2.5-flash',
    config=types.GenerateContentConfig(
        temperature=1,
        top_p=1,
        max_output_tokens=1024,
    ),
    contents=code_prompt)

Markdown(response.text)

```python
def factorial(n):
    if not isinstance(n, int) or n < 0:
        raise ValueError("Factorial is defined for non-negative integers only.")
    if n == 0:
        return 1
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result
```

### Code execution

The Gemini API can automatically run generated code too, and will return the output.

In [11]:
from pprint import pprint

config = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())],
)

code_exec_prompt = """
Generate the first 14 odd prime numbers, then calculate their sum.
"""

response = client.models.generate_content(
    model='gemini-2.5-flash',
    config=config,
    contents=code_exec_prompt)

for part in response.candidates[0].content.parts:
  pprint(part.to_json_dict())
  print("-----")

{'executable_code': {'code': 'def is_prime(n):\n'
                             '    if n < 2:\n'
                             '        return False\n'
                             '    for i in range(2, int(n**0.5) + 1):\n'
                             '        if n % i == 0:\n'
                             '            return False\n'
                             '    return True\n'
                             '\n'
                             'odd_primes = []\n'
                             'num = 3  # Start checking from 3, as 2 is the '
                             'only even prime and the user asked for odd '
                             'primes.\n'
                             'count = 0\n'
                             '\n'
                             'while count < 14:\n'
                             '    if is_prime(num):\n'
                             '        odd_primes.append(num)\n'
                             '        count += 1\n'
                             '    num

This response contains multiple parts, including an opening and closing text part that represent regular responses, an `executable_code` part that represents generated code and a `code_execution_result` part that represents the results from running the generated code.

You can explore them individually.

In [12]:
for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    elif part.executable_code:
        display(Markdown(f'```python\n{part.executable_code.code}\n```'))
    elif part.code_execution_result:
        if part.code_execution_result.outcome != 'OUTCOME_OK':
            display(Markdown(f'## Status {part.code_execution_result.outcome}'))

        display(Markdown(f'```\n{part.code_execution_result.output}\n```'))

```python
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

odd_primes = []
num = 3  # Start checking from 3, as 2 is the only even prime and the user asked for odd primes.
count = 0

while count < 14:
    if is_prime(num):
        odd_primes.append(num)
        count += 1
    num += 2 # Only check odd numbers

print(f"The first 14 odd prime numbers are: {odd_primes}")
print(f"Their sum is: {sum(odd_primes)}")
```

```
The first 14 odd prime numbers are: [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
Their sum is: 326

```

The first 14 odd prime numbers are: 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47.

Their sum is 326.

### Explaining code

The Gemini family of models can explain code to you too. In this example, you pass a [bash script](https://github.com/magicmonty/bash-git-prompt) and ask some questions.

In [13]:
file_contents = !curl https://raw.githubusercontent.com/magicmonty/bash-git-prompt/refs/heads/master/gitprompt.sh

explain_prompt = f"""
Please explain what this file does at a very high level. What is it, and why would I use it?

```
{file_contents}
```
"""

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=explain_prompt)

Markdown(response.text)

This file is a **shell script** (specifically written for `bash` and `zsh` shells) that provides a **highly customizable Git prompt**.

## What is it?

At a very high level, this file is the core logic for a tool often called "Bash Git Prompt" (or similar names). Its purpose is to **enhance your command line prompt (`PS1`)** to display real-time information about your current Git repository.

Instead of just showing your current directory, this script integrates Git status, branch names, and other relevant details directly into your prompt, making it much more informative.

## What does it do?

When sourced (loaded) into your shell (typically from your `.bashrc` or `.zshrc` file), this script does the following:

1.  **Hooks into your shell's prompt mechanism:** It modifies the `PROMPT_COMMAND` variable (in bash) or similar mechanisms (in zsh) so that a special function (`setGitPrompt`) is run *before* every new prompt is displayed.
2.  **Detects Git repositories:** When you navigate to a directory that is part of a Git repository, it detects this.
3.  **Gathers Git status:** It then runs `git` commands (e.g., `git status`, `git rev-parse`) to determine:
    *   Your current branch name.
    *   If you're in a detached HEAD state.
    *   If there are staged, unstaged, conflicting, or untracked changes.
    *   If your local branch is ahead of, behind, or has diverged from its remote counterpart.
    *   If there are stashed changes.
4.  **Constructs a rich prompt string:** Based on the gathered Git information, it dynamically creates a new `PS1` (your prompt string). This prompt typically includes:
    *   The current branch name.
    *   Symbols or counts indicating pending changes (e.g., `*` for unstaged, `+` for staged, `~` for conflicts).
    *   Indicators for ahead/behind remote (e.g., `↑` for ahead, `↓` for behind).
    *   Color-coding to make the information easy to read and distinguish.
5.  **Supports Themes and Customization:** It loads color definitions and themes from other files (like `prompt-colors.sh` and various `.bgptheme` files), allowing you to customize the look and feel of your Git prompt. You can even create your own custom themes.
6.  **Integrates with Virtual Environments:** It can also display the name of your active Python `virtualenv`, `conda` environment, or `node` virtual environment in the prompt.
7.  **Asynchronous Operations:** It includes functions (`async_run`, `async_run_zsh`) to perform non-blocking `git fetch` operations in the background, so your prompt stays updated with remote changes without slowing down your terminal.

## Why would I use it?

You would use this file to:

*   **Get Instant Git Context:** Without typing `git status`, you immediately know which branch you're on, if you have pending changes (staged, unstaged, conflicts), and your local branch's relationship to its remote.
*   **Boost Productivity:** Save time and mental effort by having critical Git information constantly visible, reducing the need to frequently run Git commands to check your status.
*   **Prevent Mistakes:** Easily see if you're on the wrong branch before committing, or if you have unpushed changes that need to be synchronized.
*   **Enhance Terminal Aesthetics:** Make your terminal more informative and visually appealing with color-coded Git details.
*   **Stay Informed about Virtual Environments:** If you work with multiple Python, Node, or Conda projects, it clearly shows which environment is currently active.

In essence, it turns your otherwise basic shell prompt into a powerful dashboard for your Git workflow.